# DATAML 2024-25: Pulse Shape Discrimination

**Objective**  
This project focuses on the discrimination between two types of scintillation signals, **Li6** and **Po**, and how the resulting model (or models) can be applied to a third, unlabeled dataset **Phys**.

Below, we walk through:
1. **Dataset Exploration**: Understanding waveform differences, preliminary statistics, and basic comparisons.
2. **Feature-Based Classification**: Basic ML classifiers (RandomForest, SVM, XGBoost).
3. **CNN-Based Approaches**: Single-branch, multi-branch, and attention-based architectures for deeper insight.
4. **Model Uncertainty & Contamination Control**: Ensuring that Po contamination remains at or below 5%.
5. **Classifying Phys**: Predicting the class labels (Li6 vs. Po) for an unlabeled real-world dataset.

In response to the **Project Tasks**, additional Markdown explanations and in-code comments have been provided to elucidate each step, methodology, and rationale.

## 1. Dataset Exploration

### Motivation
We begin by **qualitatively** exploring and visualizing the Li6 and Po datasets. This serves to reveal:
- Initial differences in shape, amplitude, frequency components, etc.
- Key features that might provide robust discrimination (mean amplitude, maximum amplitude, frequency domain patterns, etc.).

### Format of Input Data
Each signal is stored as a multi-dimensional array of shape `(N, 4, T)`, where:
- **N** = number of samples in the dataset
- **4** = number of detector channels
- **T** = number of timepoints per waveform (e.g., 2000 or truncated)

These waveforms are used both for **feature-based** ML approaches and for **deep learning** (CNNs, attention). Truncation or fraction sampling is often applied to reduce computational load.

### PyTorch Dataset & Splitting
For downstream tasks, we build a custom **PyTorch Dataset** (see `ScintillationDataset`) that returns `(waveform, label)` pairs. A typical 80/20 train-test split is used to ensure a robust evaluation while retaining sufficient training data. In some cases, we sample only a fraction of the dataset due to memory or time constraints.

## Load Libraries and Utils

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Data loading & feature extraction utilities
from utils.data_loading import load_waveforms
from utils.features_extraction import extract_features_dataset

# Waveform analysis (stats & waveforms)
from utils.waveforms_analysis import (
    compute_statistics,
    plot_stat_distributions,
    compute_average_waveform,
    plot_overall_average_overlay,
    plot_random_waveforms,
    plot_average_waveforms_per_detector,
    plot_difference_waveforms_per_detector,
    plot_heatmap_avg_difference
)

# Features visualization
from utils.features_visualization import (
    plot_features_by_base_feature
)

# Machine Learning utilities
from utils.ml_classifiers import create_dataset, train_and_evaluate_ml
# Deep Learning utilities
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from utils.deep_learning import (
    TwoDConvNet,
    train_one_epoch,
    evaluate,
    ScintillationDataset,
    TwoDConvNetWithFullAttention,
    MultiBranchCNN,
    train_and_evaluate
)

# Uncertainty estimation utilities
from utils.uncertainty import (
    mc_dropout_predict,
    plot_uncertainty_distribution,
    plot_sample_prediction
)

# Analysis utilities
from utils.analysis import (
    find_threshold_for_contamination,
    classify_dataset
)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

import pandas as pd

import xgboost

### 1.1. Loading the Data

We begin by loading a small fraction (1%) of each dataset for rapid testing and memory savings. 
Here, we specify `time_start=200` and `time_end=2000`, which effectively **truncates** the waveform
to focus on the most relevant region (often the region where the scintillation event is dominant).

In [ ]:
# Explanation: We load Li6 and Po waveforms using our helper function "load_waveforms".
# Note that "fraction=0.01" uses only 10% of the entire dataset for quick iteration.

Li6_waveforms = load_waveforms("data/Li6.npz", key="data_array", fraction=0.1, time_start=0, time_end=7756)
Po_waveforms = load_waveforms("data/Po.npz", key="data_array", fraction=0.1, time_start=0, time_end=7756)

print(f"Li6 shape: {Li6_waveforms.shape}, Po shape: {Po_waveforms.shape}")
print(f"Li6 dataset shape: {Li6_waveforms.shape} (samples, channels, timepoints)")
print(f"Po dataset shape: {Po_waveforms.shape} (samples, channels, timepoints)")

### 1.2. Random Waveform Visualization

Plotting random waveforms provides a **qualitative** look at amplitude levels, noise characteristics, and general shape. This helps us form an intuition about how Li6 and Po differ in their time-domain behavior.

In [ ]:
plot_random_waveforms(Li6_waveforms, dataset_name="Li6", num_samples=1)
plot_random_waveforms(Po_waveforms, dataset_name="Po", num_samples=1)

### 1.3. Basic Statistics

We compute distributions of **mean** and **max** amplitudes for both Li6 and Po. Such global statistics often reveal fundamental distinctions in event amplitude.

In [ ]:
# Explanation: compute_statistics returns basic stats like mean and max per sample.
Li6_stats = compute_statistics(Li6_waveforms)
Po_stats = compute_statistics(Po_waveforms)

# Plot distributions of Mean Amplitude
plot_stat_distributions(
    Li6_stats["mean"], Po_stats["mean"],
    stat_name="Mean Amplitude", dataset_names=("Li6", "Po")
)

# Plot distributions of Max Amplitude
plot_stat_distributions(
    Li6_stats["max"], Po_stats["max"],
    stat_name="Max Amplitude", dataset_names=("Li6", "Po")
)

### 1.4. Average Waveform Comparison

We visualize the **average** waveform per channel for Li6 and Po, and then examine the difference. 
This highlights which time segments or channels may exhibit the largest dissimilarities between classes.

In [ ]:
# Explanation: We aggregate waveforms sample-wise to compute channel-wise average signals for Li6 and Po.
avg_Li6 = compute_average_waveform(Li6_waveforms)
avg_Po = compute_average_waveform(Po_waveforms)

# Plot average waveforms per detector
plot_average_waveforms_per_detector(avg_Li6, avg_Po)

# Plot differences in average waveforms per detector
plot_difference_waveforms_per_detector(avg_Li6, avg_Po)

Here, we create a **2D histogram** to visualize the time vs. amplitude difference across channels.

In [ ]:
plot_heatmap_avg_difference(avg_Li6, avg_Po)

### 1.5. Focused Analysis on Initial Time Indices

Often, the primary discrimination power lies in the **initial** part of the waveform. Below, we truncate the first 800 time samples and replicate our visual analyses. This can reveal more pronounced differences if noise or baseline variations occur later in time.

In [ ]:
time_focus_start = 0
time_focus_end = 800

avg_Li6_focus = avg_Li6[:, time_focus_start:time_focus_end]
avg_Po_focus = avg_Po[:, time_focus_start:time_focus_end]

plot_average_waveforms_per_detector(avg_Li6_focus, avg_Po_focus)
plot_difference_waveforms_per_detector(avg_Li6_focus, avg_Po_focus)

plot_heatmap_avg_difference(avg_Li6_focus, avg_Po_focus)

## 2. Feature Extraction

We extract features capturing both time and frequency domain characteristics:
- **Mean**: Average amplitude over time.
- **Peak**: Maximum amplitude.
- **Dominant Frequency**: Main frequency peak in the spectrum.
- **Spectral Entropy**: Measures the distribution of spectral energy.

Each channel (4 total) contributes these 4 features, leading to 16 dimensions per sample. We then visualize these distributions across Li6 vs. Po.

In [ ]:
feature_names = ["Mean", "Peak", "Dominant_Freq", "Spectral_Entropy"]
feature_names_full = [f"{name}_Ch{i}" for i in range(4) for name in feature_names]

# Example assumption: real sampling rate is 1 MHz
Li6_features = extract_features_dataset(Li6_waveforms, sampling_rate=1000000.0)
Po_features  = extract_features_dataset(Po_waveforms, sampling_rate=1000000.0)

### 2.1. Compare Extracted Features by Base Feature

We produce histograms of each **base feature** across all four channels, overlaying Li6 vs Po. 
This allows for a channel-specific analysis, e.g., how does the mean amplitude in channel 2 compare between Li6 and Po?

In [ ]:
base_features = ["Mean", "Peak", "Dominant_Freq", "Spectral_Entropy"]

plot_features_by_base_feature(
    features_Li6=Li6_features,
    features_Po=Po_features,
    base_feature_names=base_features,
    save_path="plots/features_by_base_feature",
    show=True
)

### 2.2. Detailed Pair Plot of Selected Features

To visualize **joint distributions**, we pick 4 representative features: 
- **Mean_Ch0**, **Peak_Ch1**, **Spectral_Entropy_Ch2**, **Mean_Ch3** 
and plot a Seaborn Pair Plot. 

This helps confirm or reject correlations that might be used for discrimination.

In [ ]:
selected_features = ["Mean_Ch0", "Peak_Ch1", "Spectral_Entropy_Ch2", "Mean_Ch3"]
print(f"Selected Features for Pair Plot: {selected_features}")

import pandas as pd

# Determine indices corresponding to selected features
selected_indices = []
for feat in selected_features:
    if feat in feature_names_full:
        selected_indices.append(feature_names_full.index(feat))
    else:
        print(f"Warning: feature '{feat}' not found in 'feature_names_full'.")

# Combine Li6 and Po features
features = np.vstack([Li6_features, Po_features])
labels = np.concatenate([
    np.zeros(len(Li6_features), dtype=int),
    np.ones(len(Po_features), dtype=int)
])

# Subset the columns from 'features'
subset_features = features[:, selected_indices]

# Create a DataFrame for Seaborn
df_subset = pd.DataFrame(subset_features, columns=selected_features)
df_subset["Label"] = ["Li6" if lab == 0 else "Po" for lab in labels]

print(f"Shape of the pair-plot DataFrame: {df_subset.shape}")
print(df_subset.head())

import seaborn as sns

pair_plot = sns.pairplot(
    data=df_subset, 
    hue="Label",
    diag_kind="kde",
    corner=True,
    palette=["blue", "red"]
)

pair_plot.fig.suptitle("Pair Plot of Selected Features (Li6 vs Po)", y=1.02)
plt.show()

# 3. Building a Simple Classifier

**Motivation**: We start with simpler classifiers (RandomForest, SVM, XGBoost) as baselines before proceeding to more complex neural architectures.

- **Goal**: Achieve at least 80% accuracy in discriminating Li6 vs Po.
- **Metrics**: We log accuracy, confusion matrix, classification report, etc.
- **Stopping Criterion**: For these simpler models, we rely on standard model training or built-in solvers (e.g., XGBoost’s iteration limit). Early stopping can be used if overfitting is detected.

In [ ]:
# Explanation: The function create_dataset merges Li6 & Po features -> (X, y).
use_fusion = False  # If True, uses 'extract_features_fusion'; otherwise uses simpler approach.

X, y = create_dataset(
    Li6_data=Li6_waveforms,
    Po_data=Po_waveforms,
    fusion=use_fusion
)
print("Dataset shapes:")
print("X:", X.shape, "  y:", y.shape)
print("First row of X:", X[0])
print("First label:", y[0])

### 3.1. Train-Test Split + Normalization
We allocate 80% for training, 20% for testing, and standardize the features with `StandardScaler`. 
This helps many classifiers (like SVM) by giving each feature zero mean, unit variance.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.2, 
    random_state=42
)
print(f"Train set size: {X_train.shape}, Test set size: {X_test.shape}")

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

print("Example scaled training row:", X_train_scaled[0])

### 3.2. Model Selection and Training

We loop through three model types: 
- **Random Forest (rf)** 
- **Support Vector Machine (svm)** 
- **XGBoost (xgb)**  

We measure performance on the test set after training, visualizing a **probability histogram**, **confusion matrix**, and **classification report**. We also log final test accuracies.

In [ ]:
model_types = ["rf", "svm", "xgb"]  # "RandomForest", "SVM", and "XGBoost"
threshold = 0.5                    # Probability threshold for classification

results_summary = {}

for m_type in model_types:
    print("======================================================")
    print(f"TRAINING MODEL: {m_type.upper()}")
    print("======================================================")

    trained_model, y_pred_proba = train_and_evaluate_ml(
        X_train_scaled,
        X_test_scaled,
        y_train,
        y_test,
        model_type=m_type,
        threshold=threshold
    )
    
    # Convert probabilities -> predicted labels
    y_pred = (y_pred_proba >= threshold).astype(int)
    acc = accuracy_score(y_test, y_pred)
    results_summary[m_type] = acc

print("\nSUMMARY OF ACCURACIES:")
for k, v in results_summary.items():
    print(f"Model {k.upper()} => Accuracy = {v:.3f}")

# 4. Deep Learning Approaches

**Motivation**: CNNs and Attention can exploit the temporal structure of the waveforms more directly than feature-engineering alone.

In the code below, we illustrate:
1. **Single-Branch 2D CNN**: Treats `(4,time)` as a 2D “image.”
2. **Multi-Branch CNN**: Each channel is processed by its own 1D CNN branch, then concatenated.
3. **CNN + Attention**: Incorporates a self-attention mechanism to highlight the most relevant time segments.

**Loss & Accuracy** are recorded at each epoch. We generally train for a fixed number of epochs (e.g. 10) and would apply **early stopping** if the validation loss fails to improve (not shown here, but recommended in practice).

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


### 4.1. Data Preparation for CNN

We load Li6 and Po waveforms again (optionally truncated), label them, and create a **ScintillationDataset** for PyTorch. 
We then do an 80/20 split and build DataLoaders.

In [3]:
li6_data = load_waveforms("data/Li6.npz", key="data_array", fraction=0.2, time_start=0, time_end=7756)
po_data  = load_waveforms("data/Po.npz",  key="data_array", fraction=0.2, time_start=0, time_end=7756)
time_span = 7756
print("Li6 shape=", li6_data.shape, "Po shape=", po_data.shape)

li6_labels = np.zeros(len(li6_data), dtype=np.int64)
po_labels  = np.ones(len(po_data),  dtype=np.int64)

# Explanation: We combine the Li6 & Po waveforms, but for deep learning below,
# we typically rely on the custom 'ScintillationDataset' rather than a single array.
full_data   = np.concatenate([li6_data, po_data], axis=0)
full_labels = np.concatenate([li6_labels, po_labels], axis=0)

# Build PyTorch dataset => returns (waveform, label)
dataset = ScintillationDataset(li6_data, po_data)

ds_size     = len(dataset)
train_size  = int(0.8 * ds_size)
test_size   = ds_size - train_size
train_ds, test_ds = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_ds, batch_size=64, shuffle=False)

Li6 shape= (10600, 4, 7756) Po shape= (10447, 4, 7756)


### 4.2. Single-Branch 2D CNN

We define a simple 2D CNN (see `TwoDConvNet`) that treats each sample as `(1, 4, width)`. 
We train for 10 epochs and evaluate test accuracy/loss.

**Note**: In real experiments, you would:
 - Possibly train longer or apply early stopping
 - Tune batch size, learning rate, etc.

In [ ]:
model_cnn = TwoDConvNet(num_classes=2, width=time_span)

trained_model_cnn, metrics_cnn = train_and_evaluate(
    model=model_cnn,
    train_loader=train_loader,
    test_loader=test_loader,
    num_epochs=10,
    learning_rate=1e-3,
    device=device
)

torch.save(trained_model_cnn.state_dict(), "models/trained_model_cnn.pth")


### 4.3. Multi-Branch CNN

Each of the 4 channels is processed independently in separate 1D CNN branches, preserving each detector’s unique characteristics. The outputs are concatenated and fed into a classification head.

In [ ]:
model_mb = MultiBranchCNN(input_length=time_span, num_classes=2)

trained_model_mb, metrics_mb = train_and_evaluate(
    model=model_mb,
    train_loader=train_loader,
    test_loader=test_loader,
    num_epochs=10,
    learning_rate=1e-3,
    device=device
)

torch.save(trained_model_cnn.state_dict(), "models/trained_model_mb.pth")


### 4.4. CNN + Attention

We incorporate **self-attention** to learn a weighting over the temporal dimension, focusing on the most relevant segments. 
This is especially beneficial if only certain time intervals contain discriminative signal features.

In [ ]:
model_attn = TwoDConvNetWithFullAttention(num_classes=2, feature_size=64)

trained_model_attn, metrics_attn = train_and_evaluate(
    model=model_attn,
    train_loader=train_loader,
    test_loader=test_loader,
    num_epochs=10,
    learning_rate=1e-3,
    device=device
)

torch.save(trained_model_cnn.state_dict(), "models/trained_model_attn.pth")


## 5. Epistemic Uncertainty Estimation

**Motivation**: We seek to understand when the model is “unsure,” especially crucial in physics applications. 

**Chosen Method**: **Monte Carlo Dropout** (Gal & Ghahramani, 2016)  
By enabling dropout at inference and performing multiple stochastic forward passes, we obtain a distribution of predicted probabilities.

- **Mean**: Central predicted probability for Li6 (or Po).  
- **Standard Deviation**: Confidence or uncertainty measure.  

This helps identify ambiguous signals and refine our contamination thresholds.

In [ ]:
# Load state dictionaries from files
state_dict_attn = torch.load("models/trained_model_attn.pth", map_location=torch.device(device))
state_dict_cnn = torch.load("models/trained_model_cnn.pth", map_location=torch.device(device))  # Fixed filename
state_dict_mb = torch.load("models/trained_model_mb.pth", map_location=torch.device(device))

# Remove "module." prefix if the model was saved using DataParallel
state_dict_attn = {k.replace("module.", ""): v for k, v in state_dict_attn.items()}
state_dict_cnn = {k.replace("module.", ""): v for k, v in state_dict_cnn.items()}
state_dict_mb = {k.replace("module.", ""): v for k, v in state_dict_mb.items()}

# Load state dictionaries into respective models
trained_model_attn = TwoDConvNetWithFullAttention(num_classes=2, feature_size=64)
trained_model_cnn = TwoDConvNet(num_classes=2, width=time_span)
trained_model_mb = MultiBranchCNN(input_length=time_span, num_classes=2)

trained_model_attn.load_state_dict(state_dict_attn)
trained_model_cnn.load_state_dict(state_dict_cnn)
trained_model_mb.load_state_dict(state_dict_mb)

print("Models loaded successfully.")

# Ensure all models are on the same device as inputs
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

trained_model_attn = trained_model_attn.to(device)
trained_model_cnn = trained_model_cnn.to(device)
trained_model_mb = trained_model_mb.to(device)

# Ensure models are in evaluation mode
trained_model_attn.eval()
trained_model_cnn.eval()
trained_model_mb.eval()

# Prepare models for evaluation
models = [
    (trained_model_attn, "CNN+Attention"),
    (trained_model_cnn,  "Single-Branch CNN"),
    (trained_model_mb,   "Multi-Branch CNN")
]

# Perform predictions with MC Dropout
for model, model_name in models:
    predictions_mean, predictions_std, true_labels = mc_dropout_predict(
        model=model,
        dataloader=test_loader,
        device=device,  # Ensure the model and data loader use the same device
        num_forward_passes=30
    )

    # Plot the overall distribution of standard deviations
    plot_uncertainty_distribution(
        predictions_mean, 
        predictions_std, 
        bins=40,
        title=f"{model_name}: Uncertainty Distribution"
    )

    # Inspect an example sample's predictive distribution
    sample_to_check = 10
    plot_sample_prediction(
        predictions_mean, 
        predictions_std, 
        sample_idx=sample_to_check, 
        class_names=["Li6", "Po"]
    )



C:\Users\André\AppData\Local\Temp\ipykernel_12308\984705833.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict_attn = torch.load("models/trained_model_attn.pth",

Models loaded successfully.


## 6. Limit Po Contamination to 5%

In many real-world scenarios, we have a **hard limit** on Po contamination. We choose a probability threshold on **p(Li6)** that ensures only 5% of actual Po signals are misclassified as Li6.

This approach might reduce total Li6 recall, but maintains the contamination requirement.

In [ ]:
thresholds = {}

models = [
    (trained_model_cnn,  "Single-Branch CNN"),
    (trained_model_mb,   "Multi-Branch CNN"),
    (trained_model_attn, "CNN+Attention")
]

# Explanation: "mc_dropout_predict" can be used for single pass as well (num_forward_passes=1).
# We then pass these predicted Li6 probabilities to a function that solves for the threshold that yields max_po_contamination <= 5%.

for model, name in models:
    predictions_mean, _, true_labels = mc_dropout_predict(
        model=model,
        dataloader=test_loader,     
        device=device,
        num_forward_passes=1        
    )

    p_li6 = predictions_mean[:, 0]
    max_po_contamination = 0.05
    threshold = find_threshold_for_contamination(
        probabilities_li6=p_li6,
        labels=true_labels,         # 0=Li6, 1=Po
        max_po_contamination=max_po_contamination
    )

    if threshold is None:
        print(f"{name}: Could NOT find threshold for 5% contamination.")
        threshold = 0.5
    else:
        print(f"{name}: threshold for 5% contamination = {threshold:.3f}")

    thresholds[name] = threshold

    predicted_labels = np.where(p_li6 > threshold, 0, 1)  # 0=Li6, 1=Po

    li6_mask = (true_labels == 0)
    po_mask  = (true_labels == 1)

    li6_retained = np.sum((predicted_labels[li6_mask] == 0)) / np.sum(li6_mask)
    po_mislabeled = np.sum((predicted_labels[po_mask] == 0)) / np.sum(po_mask)

    print(f"{name} => Li6 retained: {li6_retained*100:.2f}% | "
          f"Po contamination: {po_mislabeled*100:.2f}%")
    print("-"*60)

## 7. Classify the Phys Dataset

Finally, we apply the trained models to an unlabeled **Phys** dataset. Using the threshold found above (which ensures ≤5% Po contamination), we assign each signal to Li6 or Po. We then estimate how many signals are Li6 vs. Po.

In [ ]:
phys_waveforms = load_waveforms(
    file_path="data/Phys.npz",
    fraction=1,         
    time_start=0,
    time_end=7756
)
print("Phys dataset shape:", phys_waveforms.shape)

for model, name in models:
    best_thresh = thresholds[name]
    predicted_labels_phys = classify_dataset(
        model=model,
        waveforms=phys_waveforms,
        device=device,
        threshold=best_thresh
    )

    li6_count = np.sum(predicted_labels_phys == 0)
    po_count  = np.sum(predicted_labels_phys == 1)
    total     = len(predicted_labels_phys)

    print(f"\nModel: {name}")
    print(f"Threshold used: {best_thresh:.3f}")
    print(f"Estimated Li6 in Phys: {li6_count} ({li6_count/total*100:.2f}%)")
    print(f"Estimated Po  in Phys: {po_count}  ({po_count/total*100:.2f}%)")
    print("-"*60)

# Another Approach

During our testing, we noticed that most Po had signal after 4000 while almost none Si. From this simple idea, we developped a 10 lines script that has a similar accuracy to our best CNN.

In [5]:
Po = np.load("data/Po.npz")
Pow= Po['data_array']
print(Pow.shape)

Li6 = np.load("data/Li6.npz")
Liw= Li6['data_array']
print(Liw.shape)


Phys = np.load("data/Phys.npz")
Physw= Phys['data_array']
print(Physw.shape)


# Initialization of counters for Po and Li
count_Po = 0
count_Li = 0
threshold = 6000  # Define the threshold


for signal in Pow:
    # Count the number of channels with non-zero values after the threshold
    valid_channels = sum([np.any(signal[channel][threshold:] != 0) for channel in range(4)])
    if valid_channels >= 2:
        count_Po += 1

for signal in Liw:
    valid_channels = sum([np.any(signal[channel][threshold:] != 0) for channel in range(4)])
    if valid_channels >= 2:
        count_Li += 1

# Total of Po and Li events
nb_Po = Pow.shape[0]
nb_Li = Liw.shape[0]

# Calculate percentages of valid and invalid signals
nb_Po_OK = count_Po / nb_Po
nb_Po_notOK = 1 - nb_Po_OK
nb_Li_notOK = count_Li / nb_Li
nb_Li_OK = 1 - nb_Li_notOK

print(f"Percentage of valid Po signals: {nb_Po_OK * 100:.2f}%")
print(f"Percentage of invalid Po signals: {nb_Po_notOK * 100:.2f}%")
print(f"Percentage of valid Li signals: {nb_Li_OK * 100:.2f}%")
print(f"Percentage of invalid Li signals: {nb_Li_notOK * 100:.2f}%")


# Create the confusion matrix
conf_mat = np.array([[count_Po, nb_Po - count_Po], 
                     [count_Li, nb_Li- count_Li]])
conf_mat = conf_mat.astype(int)

plt.figure(figsize=(6, 6))
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues', xticklabels=['Po', 'Li'], yticklabels=['Po', 'Li'])
plt.title("Confusion Matrix")
plt.ylabel("True Labels")
plt.xlabel("Predicted Labels")
plt.show()

(52238, 4, 7756)
(53000, 4, 7756)
(24000, 4, 7756)


KeyboardInterrupt: 

We can recount the number of Si

In [ ]:
count_Phys = 0
# Iterate over the Phys signals
for signal in Physw:
    # Count the number of channels with non-zero values after the threshold
    valid_channels = sum([np.any(signal[channel][threshold:] != 0) for channel in range(4)])
    # Increment the counter if at least 2 channels meet the criterion
    if valid_channels >= 2:
        count_Phys += 1

print(f"Number of Po in Phys: {count_Phys}")
print(f"Number of Li in Phys: {Physw.shape[0] - count_Phys}")


As seen above, the results obtained using this method are extremely encouraging: less than 5% false positives for both Li and Po. This provides essential insights for developing a machine learning-based solution: all signals are useful for discriminating between Li and Po, not just the portion containing the main pulse. Furthermore, it would be interesting to explore a multi-branch method that considers all four signals simultaneously, rather than treating them as four independent signals.

In [ ]:
# Initialization of counters for Po and Li
thresholds = np.linspace(2000, 7000, 10, dtype=int)  # Thresholds ranging from 4000 to 7000 with 10 points
Po_percentages = []
Li_percentages = []

# Iterate over different thresholds
for threshold in thresholds:
    count_Po = 0
    count_Li = 0
    
    # Count valid signals for Po
    for signal in Pow:
        valid_channels = sum([np.any(signal[channel][threshold:] != 0) for channel in range(4)])
        if valid_channels >= 2:
            count_Po += 1

    # Count valid signals for Li
    for signal in Liw:
        valid_channels = sum([np.any(signal[channel][threshold:] != 0) for channel in range(4)])
        if valid_channels >= 2:
            count_Li += 1

    # Total of Po and Li events
    nb_Po = Pow.shape[0]
    nb_Li = Liw.shape[0]

    # Calculate percentages of valid signals for this threshold
    nb_Po_OK = count_Po / nb_Po
    nb_Li_notOK = count_Li / nb_Li
    nb_Li_OK = 1 - nb_Li_notOK


    # Store the percentages
    Po_percentages.append(nb_Po_OK * 100)
    Li_percentages.append(nb_Li_OK * 100)

# Plot Percentages against thresholds
sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))
plt.plot(thresholds, Po_percentages, marker='o', label="Po Percentages")
plt.plot(thresholds, Li_percentages, marker='o', label="Li Percentages")
plt.title("Percentages of Valid Signals vs Thresholds")
plt.xlabel("Thresholds")
plt.ylabel("Percentage (%)")
plt.legend()
plt.show()


# Project Tasks Answers

Below is a concise summary of how the **above code** addresses the five requested tasks:

---
**1. Explore the datasets & qualitative differences**  
- *Qualitative method*:  
  We plotted **random waveforms** and **average waveforms** for Li6 vs Po, then compared distributions of **mean** and **max** amplitude. Histograms and difference plots highlight key discrepancies (e.g., Li6 sometimes exhibits a lower amplitude in certain channels).  
- *Chosen input format*:  
  Our data shape is `(N, 4, T)`, with 4 channels and T timepoints. This format allows direct input to CNNs and simpler 1D or 2D transformations.  
- *PyTorch Dataset & splits*:  
  We defined `ScintillationDataset` (in `utils.deep_learning`) to yield `(waveform, label)` pairs. A standard 80/20 split is used for training/test.  

---
**2. Build a classifier model & aim >80% accuracy**  
- We tested several models (RandomForest, SVM, XGBoost) using *feature engineering* (Mean, Peak, etc.) and later advanced **CNN** models (single-branch, multi-branch, attention).  
- *Training metrics*: We log the **loss** and **accuracy** per epoch in the deep-learning approach, and confusion matrices for the simpler ML models.  
- *Stopping rationale*: We ran a fixed 10-epoch training for CNNs, acknowledging that in practice, we might use early stopping if the validation loss plateaus or deteriorates.  

---
**3. Estimate epistemic uncertainty**  
- *Method chosen*: **Monte Carlo Dropout**. It is straightforward to implement, requires multiple forward passes at inference, and yields a distribution of predicted probabilities.  

---
**4. Limit Po contamination to 5%**  
- Using the model’s Li6 probability `p(Li6)`, we scanned for a *threshold* such that only 5% of Po signals exceed it (i.e., are misidentified as Li6). This ensures contamination is capped at 5%.  
- We then observed how many Li6 events are *retained* at that threshold.  

---
**5. Classify Phys**  
- With the threshold set to meet the contamination requirement, we predicted Li6 vs Po for the **Phys** dataset. We reported the counts (and percentages) of each.  

---

*End of Notebook*